In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

import numpy as np

import pyconll

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset

import simcube
from simcube.data import \
    tokenize_corpus, \
    build_vocabulary, \
    character_tokenize, \
    pos_corpus_to_tensor, \
    POSTagger
from simcube.pipeline import \
    train_eval_loop, \
    predict_with_model, \
    init_random_seed

init_random_seed()

## Загрузка текстов и разбиение на обучающую и тестовую подвыборки

In [2]:
full_train = pyconll.load_from_file('../../data/raw/rus_tag_corpus/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('../../data/raw/rus_tag_corpus/ru_syntagrus-ud-dev.conllu')

In [3]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print("\n")

Анкета NOUN
. PUNCT


Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT




In [4]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [5]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [6]:
train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, max_doc_freq=1.0, min_count=5, pad_word='<PAD>')
print("Количество уникальных символов", len(char_vocab))
print(list(char_vocab.items())[:10])

Количество уникальных символов 150
[('<PAD>', 0), (' ', 1), ('о', 2), ('е', 3), ('а', 4), ('т', 5), ('и', 6), ('н', 7), ('.', 8), ('с', 9)]


In [7]:
char_vocab

{'<PAD>': 0,
 ' ': 1,
 'о': 2,
 'е': 3,
 'а': 4,
 'т': 5,
 'и': 6,
 'н': 7,
 '.': 8,
 'с': 9,
 'р': 10,
 'л': 11,
 'в': 12,
 'к': 13,
 'д': 14,
 'м': 15,
 'п': 16,
 'у': 17,
 'ы': 18,
 'я': 19,
 'ь': 20,
 'з': 21,
 'г': 22,
 'б': 23,
 ',': 24,
 'ч': 25,
 'й': 26,
 'ж': 27,
 'х': 28,
 'ш': 29,
 'ю': 30,
 'ц': 31,
 '-': 32,
 'щ': 33,
 'э': 34,
 'ф': 35,
 'В': 36,
 'П': 37,
 '"': 38,
 'Н': 39,
 'С': 40,
 'К': 41,
 'О': 42,
 'И': 43,
 'М': 44,
 'А': 45,
 'Р': 46,
 'Т': 47,
 ':': 48,
 'Д': 49,
 '0': 50,
 'Г': 51,
 '1': 52,
 'Б': 53,
 'Е': 54,
 '?': 55,
 ')': 56,
 '(': 57,
 '2': 58,
 'Э': 59,
 'У': 60,
 'З': 61,
 'Л': 62,
 '5': 63,
 'ъ': 64,
 'Ч': 65,
 'Ф': 66,
 '9': 67,
 '3': 68,
 '!': 69,
 'Я': 70,
 'Ш': 71,
 '4': 72,
 '8': 73,
 'Х': 74,
 '6': 75,
 '_': 76,
 '7': 77,
 'Ц': 78,
 '…': 79,
 'e': 80,
 '%': 81,
 'a': 82,
 'r': 83,
 'o': 84,
 'i': 85,
 'I': 86,
 'Ж': 87,
 'n': 88,
 't': 89,
 's': 90,
 'Ю': 91,
 'l': 92,
 ';': 93,
 'u': 94,
 'X': 95,
 'ё': 96,
 'c': 97,
 'm': 98,
 'M': 99,
 'd': 

In [8]:
UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}
label2id

{'<NOTAG>': 0,
 'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'PUNCT': 13,
 'SCONJ': 14,
 'SYM': 15,
 'VERB': 16,
 'X': 17}

In [9]:
train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
train_dataset = TensorDataset(train_inputs, train_labels)

test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
test_dataset = TensorDataset(test_inputs, test_labels)

In [10]:
train_inputs[1][:5]

tensor([[ 0, 39,  4, 25,  4, 11, 20,  7,  6, 13,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  2, 23, 11,  4,  9,  5,  7,  2, 22,  2,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 17, 16, 10,  4, 12, 11,  3,  7,  6, 19,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  9, 12, 19, 21,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 40,  3, 15,  3,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0

In [11]:
train_labels[1]

tensor([ 8,  1,  8,  8, 12, 12,  4,  8,  1, 13, 16,  2,  8,  3,  3, 13, 16,  2,
         8,  2,  8,  5,  3, 10, 16,  2,  8,  8,  2,  8, 13,  8, 13, 13,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0])

## Вспомогательная свёрточная архитектура

In [12]:
class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
        super().__init__()
        layers = []
        for _ in range(layers_n):
            layers.append(nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                nn.Dropout(dropout),
                nn.LeakyReLU()))
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers:
            x = x + layer(x)
        return x

## Предсказание частей речи на уровне отдельных токенов

In [22]:
# Эта сеть не использует контекст токенов, в предсказании части речи участвуют только символы самого токена

In [14]:
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [15]:
single_token_model = SingleTokenPOSTagger(len(char_vocab), len(label2id), embedding_size=64, layers_n=3, kernel_size=3, dropout=0.3)
print('Количество параметров', sum(np.product(t.shape) for t in single_token_model.parameters()))

Количество параметров 47826


In [16]:
%%time

best_val_loss, best_single_token_model = train_eval_loop(single_token_model, train_dataset, test_dataset,
    F.cross_entropy,
    lr=5e-3,
    epoch_n=10,
    batch_size=64,
    device='cuda',
    early_stopping_patience=5,
    max_batches_per_epoch_train=500,
    max_batches_per_epoch_val=100,
    lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2, factor=0.5, verbose=True)
)

Выбранное устройство: cuda
Эпоха 0
Эпоха: 501 итераций, 201.55 сек
Среднее значение функции потерь на обучении 0.08007763111276064
Среднее значение функции потерь на валидации 0.03311723077872602
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 202.19 сек
Среднее значение функции потерь на обучении 0.02833438031152337
Среднее значение функции потерь на валидации 0.026528122825640262
Новая лучшая модель!

Эпоха 2
Эпоха: 501 итераций, 197.15 сек
Среднее значение функции потерь на обучении 0.024013424901934918
Среднее значение функции потерь на валидации 0.024688546093973782
Новая лучшая модель!

Эпоха 3
Эпоха: 501 итераций, 197.56 сек
Среднее значение функции потерь на обучении 0.02207319940181966
Среднее значение функции потерь на валидации 0.021894465412686366
Новая лучшая модель!

Эпоха 4
Эпоха: 501 итераций, 197.53 сек
Среднее значение функции потерь на обучении 0.021079481579363346
Среднее значение функции потерь на валидации 0.02251679018729984

Эпоха 5
Эпоха: 501 итераций, 197.7

In [17]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
torch.save(best_single_token_model.state_dict(), '../../data/models/single_token_pos.pth')

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
single_token_model.load_state_dict(torch.load('../../data/models/single_token_pos.pth'))

In [18]:
train_pred = predict_with_model(single_token_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(single_token_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

  0%|          | 4/1525.4375 [00:00<00:41, 36.42it/s]

Выбранное устройство: cuda


1526it [00:40, 38.12it/s]                               


Среднее значение функции потерь на обучении 0.01657356694340706


  2%|▏         | 4/205.75 [00:00<00:05, 36.91it/s]

              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   9136391
         ADJ       0.88      0.94      0.91     85589
         ADP       1.00      0.99      0.99     81963
         ADV       0.87      0.91      0.89     44101
         AUX       0.85      0.75      0.80      7522
       CCONJ       0.87      0.99      0.93     30432
         DET       0.85      0.81      0.83     21968
        INTJ       0.78      0.18      0.29        78
        NOUN       0.98      0.92      0.95    214497
         NUM       0.97      0.93      0.95     13746
        PART       0.99      0.76      0.86     26651
        PRON       0.88      0.85      0.87     38438
       PROPN       0.79      0.96      0.87     32401
       PUNCT       1.00      1.00      1.00    157989
       SCONJ       0.80      0.88      0.84     16219
         SYM       1.00      0.99      1.00       840
        VERB       0.92      0.94      0.93     97670
           X       0.94    

100%|██████████| 206/205.75 [00:05<00:00, 39.31it/s]


Среднее значение функции потерь на валидации 0.018714211881160736
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.87      0.93      0.90     11222
         ADP       1.00      0.99      0.99     10585
         ADV       0.87      0.91      0.89      6165
         AUX       0.83      0.81      0.82      1106
       CCONJ       0.88      0.99      0.93      4410
         DET       0.83      0.80      0.81      3085
        INTJ       1.00      0.09      0.17        11
        NOUN       0.98      0.92      0.95     27974
         NUM       0.96      0.90      0.93      1829
        PART       0.99      0.76      0.86      3877
        PRON       0.88      0.83      0.86      5598
       PROPN       0.79      0.95      0.86      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.77      0.90      0.83      2258
         SYM       1.00      0.96      0.98        53
        VERB   

## Предсказание частей речи на уровне предложений (с учётом контекста)

In [23]:
class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [24]:
sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                              single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3),
                                              context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

Количество параметров 84882


In [25]:
%%time

best_val_loss, best_sentence_level_model = train_eval_loop(sentence_level_model, train_dataset, test_dataset,
    F.cross_entropy,
    lr=5e-3,
    epoch_n=10,
    batch_size=64,
    device='cuda',
    early_stopping_patience=5,
    max_batches_per_epoch_train=500,
    max_batches_per_epoch_val=100,
    lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2, factor=0.5, verbose=True)
)

Выбранное устройство: cuda
Эпоха 0
Эпоха: 501 итераций, 203.88 сек
Среднее значение функции потерь на обучении 0.07464262613740391
Среднее значение функции потерь на валидации 0.026716668287863825
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 203.05 сек
Среднее значение функции потерь на обучении 0.024460480539146772
Среднее значение функции потерь на валидации 0.020085257736246774
Новая лучшая модель!

Эпоха 2
Эпоха: 501 итераций, 204.39 сек
Среднее значение функции потерь на обучении 0.020006924592508767
Среднее значение функции потерь на валидации 0.016401978292603894
Новая лучшая модель!

Эпоха 3
Эпоха: 501 итераций, 202.78 сек
Среднее значение функции потерь на обучении 0.017660559854003603
Среднее значение функции потерь на валидации 0.014658478766421575
Новая лучшая модель!

Эпоха 4
Эпоха: 501 итераций, 202.77 сек
Среднее значение функции потерь на обучении 0.01652661173933459
Среднее значение функции потерь на валидации 0.014009611044853631
Новая лучшая модель!

Эпоха 5
Эп

In [28]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
torch.save(best_sentence_level_model.state_dict(), '../../data/models/sentence_level_pos.pth')

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
sentence_level_model.load_state_dict(torch.load('../../data/models/sentence_level_pos.pth'))

In [29]:
train_pred = predict_with_model(sentence_level_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

  0%|          | 4/1525.4375 [00:00<00:44, 34.29it/s]

Выбранное устройство: cuda


1526it [00:40, 37.88it/s]                               


Среднее значение функции потерь на обучении 0.01012236624956131


  2%|▏         | 4/205.75 [00:00<00:05, 36.07it/s]

              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   9136391
         ADJ       0.93      0.94      0.94     85589
         ADP       1.00      0.98      0.99     81963
         ADV       0.91      0.92      0.92     44101
         AUX       0.90      0.92      0.91      7522
       CCONJ       0.94      0.98      0.96     30432
         DET       0.92      0.92      0.92     21968
        INTJ       0.86      0.24      0.38        78
        NOUN       0.97      0.97      0.97    214497
         NUM       0.96      0.95      0.95     13746
        PART       0.97      0.88      0.92     26651
        PRON       0.95      0.93      0.94     38438
       PROPN       0.92      0.97      0.95     32401
       PUNCT       1.00      1.00      1.00    157989
       SCONJ       0.90      0.92      0.91     16219
         SYM       1.00      1.00      1.00       840
        VERB       0.95      0.97      0.96     97670
           X       0.84    

100%|██████████| 206/205.75 [00:05<00:00, 38.65it/s]


Среднее значение функции потерь на валидации 0.011377825401723385
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.93      0.93      0.93     11222
         ADP       1.00      0.99      0.99     10585
         ADV       0.92      0.91      0.91      6165
         AUX       0.88      0.91      0.90      1106
       CCONJ       0.94      0.98      0.96      4410
         DET       0.90      0.91      0.91      3085
        INTJ       1.00      0.27      0.43        11
        NOUN       0.97      0.96      0.97     27974
         NUM       0.95      0.94      0.94      1829
        PART       0.97      0.88      0.92      3877
        PRON       0.94      0.92      0.93      5598
       PROPN       0.91      0.97      0.94      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.88      0.91      0.90      2258
         SYM       1.00      0.98      0.99        53
        VERB   

In [13]:
class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.bn1 = nn.BatchNorm1d(embedding_size)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.bn2 = nn.BatchNorm1d(embedding_size)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.bn1(self.single_token_backbone(char_embeddings))
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.bn2(self.context_backbone(token_features))  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [14]:
sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                              single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3),
                                              context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

Количество параметров 85138


In [17]:
%%time

best_val_loss, best_sentence_level_model = train_eval_loop(sentence_level_model, train_dataset, test_dataset,
    F.cross_entropy,
    lr=5e-3,
    epoch_n=20,
    batch_size=128,
    device='cuda',
    early_stopping_patience=5,
    max_batches_per_epoch_train=500,
    max_batches_per_epoch_val=100,
    lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2, factor=0.5, verbose=True)
)

Выбранное устройство: cuda
Эпоха 0
Эпоха: 382 итераций, 317.19 сек
Среднее значение функции потерь на обучении 0.04765604019009006
Среднее значение функции потерь на валидации 0.017542734563064117
Новая лучшая модель!

Эпоха 1
Эпоха: 382 итераций, 315.85 сек
Среднее значение функции потерь на обучении 0.017847579177371495
Среднее значение функции потерь на валидации 0.012863060074428527
Новая лучшая модель!

Эпоха 2
Эпоха: 382 итераций, 315.26 сек
Среднее значение функции потерь на обучении 0.015498362813164426
Среднее значение функции потерь на валидации 0.01461146050132811

Эпоха 3
Эпоха: 382 итераций, 315.53 сек
Среднее значение функции потерь на обучении 0.014310227353515894
Среднее значение функции потерь на валидации 0.03934819623827934

Эпоха 4
Эпоха: 382 итераций, 315.19 сек
Среднее значение функции потерь на обучении 0.013505877546615001
Среднее значение функции потерь на валидации 0.08965974773925084
Epoch     5: reducing learning rate of group 0 to 2.5000e-03.

Эпоха 5
Эпоха

In [19]:
train_pred = predict_with_model(sentence_level_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

  0%|          | 4/1525.4375 [00:00<00:44, 34.27it/s]

Выбранное устройство: cuda


1526it [00:42, 35.99it/s]                               


Среднее значение функции потерь на обучении 0.06275895237922668


  2%|▏         | 4/205.75 [00:00<00:05, 34.34it/s]

              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   9136391
         ADJ       0.96      0.94      0.95     85589
         ADP       1.00      1.00      1.00     81963
         ADV       0.93      0.94      0.93     44101
         AUX       0.86      0.97      0.91      7522
       CCONJ       0.95      0.97      0.96     30432
         DET       0.92      0.93      0.93     21968
        INTJ       0.94      0.21      0.34        78
        NOUN       0.98      0.98      0.98    214497
         NUM       0.96      0.96      0.96     13746
        PART       0.94      0.93      0.93     26651
        PRON       0.96      0.93      0.95     38438
       PROPN       0.96      0.96      0.96     32401
       PUNCT       1.00      1.00      1.00    157989
       SCONJ       0.88      0.98      0.92     16219
         SYM       1.00      1.00      1.00       840
        VERB       0.97      0.96      0.96     97670
           X       0.95    

100%|██████████| 206/205.75 [00:05<00:00, 36.45it/s]


Среднее значение функции потерь на валидации 0.06050407141447067
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.96      0.93      0.94     11222
         ADP       1.00      1.00      1.00     10585
         ADV       0.92      0.93      0.92      6165
         AUX       0.86      0.95      0.91      1106
       CCONJ       0.95      0.97      0.96      4410
         DET       0.91      0.92      0.91      3085
        INTJ       0.60      0.27      0.37        11
        NOUN       0.98      0.98      0.98     27974
         NUM       0.95      0.95      0.95      1829
        PART       0.93      0.93      0.93      3877
        PRON       0.96      0.92      0.94      5598
       PROPN       0.96      0.94      0.95      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.86      0.97      0.91      2258
         SYM       1.00      1.00      1.00        53
        VERB    

In [20]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
torch.save(best_sentence_level_model.state_dict(), '../../data/models/sentence_level_pos.pth')

In [22]:
sentence_level_pos_tagger = POSTagger(sentence_level_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)

In [23]:
test_sentences = [
    'Мама мыла раму.',
    'Косил косой косой косой.',
    'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
    'Сяпала Калуша с Калушатами по напушке.',
    'Пирожки поставлены в печь, мама любит печь.',
    'Ведро дало течь, вода стала течь.',
    'Три да три, будет дырка.',
    'Три да три, будет шесть.',
    'Сорок сорок'
]
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

In [24]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, sentence_level_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 72.38it/s]                     

Выбранное устройство: cuda
мама-NOUN мыла-VERB раму-NOUN

косил-VERB косой-NOUN косой-NOUN косой-NOUN

глокая-ADJ куздра-NOUN штеко-NOUN будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-NOUN

ведро-ADV дало-VERB течь-NOUN вода-NOUN стала-VERB течь-NOUN

три-NUM да-CCONJ три-NUM будет-AUX дырка-NOUN

три-NUM да-CCONJ три-NUM будет-AUX шесть-NUM

сорок-NOUN сорок-NOUN



## Применение полученных теггеров и сравнение

In [21]:
single_token_pos_tagger = POSTagger(single_token_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
sentence_level_pos_tagger = POSTagger(sentence_level_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)

NameError: name 'single_token_model' is not defined

In [31]:
test_sentences = [
    'Мама мыла раму.',
    'Косил косой косой косой.',
    'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
    'Сяпала Калуша с Калушатами по напушке.',
    'Пирожки поставлены в печь, мама любит печь.',
    'Ведро дало течь, вода стала течь.',
    'Три да три, будет дырка.',
    'Три да три, будет шесть.',
    'Сорок сорок'
]
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

In [32]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, single_token_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 113.53it/s]                    

Выбранное устройство: cuda
мама-NOUN мыла-VERB раму-NOUN

косил-NOUN косой-NOUN косой-NOUN косой-NOUN

глокая-ADJ куздра-NOUN штеко-ADJ будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-VERB мама-NOUN любит-VERB печь-VERB

ведро-ADV дало-VERB течь-VERB вода-NOUN стала-VERB течь-VERB

три-NUM да-CCONJ три-NUM будет-VERB дырка-NOUN

три-NUM да-CCONJ три-NUM будет-VERB шесть-NUM

сорок-NOUN сорок-NOUN



In [33]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, sentence_level_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 111.45it/s]                    

Выбранное устройство: cuda
мама-NOUN мыла-VERB раму-NOUN

косил-VERB косой-ADJ косой-ADJ косой-NOUN

глокая-ADJ куздра-NOUN штеко-NOUN будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-VERB мама-NOUN любит-VERB печь-VERB

ведро-NOUN дало-VERB течь-VERB вода-NOUN стала-VERB течь-VERB

три-NUM да-CCONJ три-NUM будет-AUX дырка-NOUN

три-NUM да-CCONJ три-NUM будет-AUX шесть-NUM

сорок-NOUN сорок-NOUN



## Свёрточный модуль своими руками

In [34]:
class MyConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(in_channels * kernel_size, out_channels) / (in_channels * kernel_size),
                                   requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(out_channels), requires_grad=True)
    
    def forward(self, x):
        """x - BatchSize x InChannels x SequenceLen"""

        batch_size, src_channels, sequence_len = x.shape        
        if self.padding > 0:
            pad = x.new_zeros(batch_size, src_channels, self.padding)
            x = torch.cat((pad, x, pad), dim=-1)
            sequence_len = x.shape[-1]

        chunks = []
        chunk_size = sequence_len - self.kernel_size + 1
        for offset in range(self.kernel_size):
            chunks.append(x[:, :, offset:offset + chunk_size])

        in_features = torch.cat(chunks, dim=1)  # BatchSize x InChannels * KernelSize x ChunkSize
        in_features = in_features.permute(0, 2, 1)  # BatchSize x ChunkSize x InChannels * KernelSize
        out_features = torch.bmm(in_features, self.weight.unsqueeze(0).expand(batch_size, -1, -1)) + self.bias.unsqueeze(0).unsqueeze(0)
        out_features = out_features.permute(0, 2, 1)  # BatchSize x OutChannels x ChunkSize
        return out_features

In [35]:
sentence_level_model_my_conv = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                                      single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d),
                                                      context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model_my_conv.parameters()))

Количество параметров 84882


In [36]:
%%time


best_val_loss, best_sentence_level_model_my_conv = train_eval_loop(sentence_level_model_my_conv, train_dataset, test_dataset,
    F.cross_entropy,
    lr=5e-3,
    epoch_n=10,
    batch_size=64,
    device='cuda',
    early_stopping_patience=5,
    max_batches_per_epoch_train=500,
    max_batches_per_epoch_val=100,
    lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2, factor=0.5, verbose=True)
)

Выбранное устройство: cuda
Эпоха 0
Эпоха: 501 итераций, 129.90 сек
Среднее значение функции потерь на обучении 0.07622694853923753
Среднее значение функции потерь на валидации 0.018297371851040585
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 129.31 сек
Среднее значение функции потерь на обучении 0.020714238068523877
Среднее значение функции потерь на валидации 0.015549330317442961
Новая лучшая модель!

Эпоха 2
Эпоха: 501 итераций, 129.24 сек
Среднее значение функции потерь на обучении 0.018027356256937195
Среднее значение функции потерь на валидации 0.013457047960508874
Новая лучшая модель!

Эпоха 3
Эпоха: 501 итераций, 130.54 сек
Среднее значение функции потерь на обучении 0.016559043397268134
Среднее значение функции потерь на валидации 0.013518566689868965

Эпоха 4
Эпоха: 501 итераций, 130.10 сек
Среднее значение функции потерь на обучении 0.015718446603926
Среднее значение функции потерь на валидации 0.012259186300827136
Новая лучшая модель!

Эпоха 5
Эпоха: 501 итераций, 129.

In [37]:
train_pred = predict_with_model(best_sentence_level_model_my_conv, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(best_sentence_level_model_my_conv, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

  0%|          | 3/1525.4375 [00:00<01:02, 24.32it/s]

Выбранное устройство: cuda


1526it [00:59, 25.54it/s]                               


Среднее значение функции потерь на обучении 0.009719030931591988


  1%|▏         | 3/205.75 [00:00<00:08, 23.92it/s]

              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   9136391
         ADJ       0.93      0.94      0.93     85589
         ADP       1.00      0.99      1.00     81963
         ADV       0.92      0.91      0.91     44101
         AUX       0.87      0.96      0.91      7522
       CCONJ       0.94      0.98      0.96     30432
         DET       0.92      0.92      0.92     21968
        INTJ       0.80      0.41      0.54        78
        NOUN       0.98      0.96      0.97    214497
         NUM       0.96      0.96      0.96     13746
        PART       0.96      0.88      0.92     26651
        PRON       0.95      0.92      0.94     38438
       PROPN       0.93      0.97      0.95     32401
       PUNCT       1.00      1.00      1.00    157989
       SCONJ       0.89      0.92      0.90     16219
         SYM       1.00      1.00      1.00       840
        VERB       0.94      0.97      0.95     97670
           X       0.91    

100%|██████████| 206/205.75 [00:08<00:00, 25.49it/s]


Среднее значение функции потерь на валидации 0.01101172436028719
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.92      0.93      0.93     11222
         ADP       1.00      0.99      1.00     10585
         ADV       0.92      0.89      0.91      6165
         AUX       0.86      0.96      0.91      1106
       CCONJ       0.94      0.98      0.96      4410
         DET       0.90      0.91      0.90      3085
        INTJ       1.00      0.27      0.43        11
        NOUN       0.97      0.96      0.97     27974
         NUM       0.95      0.95      0.95      1829
        PART       0.96      0.88      0.92      3877
        PRON       0.94      0.91      0.93      5598
       PROPN       0.92      0.96      0.94      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.86      0.91      0.88      2258
         SYM       1.00      1.00      1.00        53
        VERB    